In [8]:
import fullcontrol as fc
from pint import UnitRegistry
u = UnitRegistry()

In [9]:
#http://www.aardvarkmastering.com/riaa.htm

In [10]:
import soundfile as sf
data, samplerate = sf.read('nightfall-future-bass-music-228100.mp3')
samplerate *= u.Hz

SongLen = len(data)/samplerate

RECORD_SPEED = 33*u.rpm

turns = (RECORD_SPEED*SongLen).to_base_units()

In [11]:
NUM_POINTS = len(data)
OUTER_RADIUS = (5.75*u.inch).to(u.mm).magnitude #inch
INNER_RADIUS = (2.25*u.inch).to(u.mm).magnitude 
MIN_EXTRUSION_WIDTH = .4
MAX_EXTRUSION_WIDTH = 1.5

In [12]:
import numpy as np
def getSteps(data, 
             turns, 
             outerRad,
             innerRad,
             minExtrusionWidth = MIN_EXTRUSION_WIDTH,
             maxExtrusionWidth = MAX_EXTRUSION_WIDTH):
    steps = []
    data = np.array(data)
    scaledData = (data-min(data))/(max(data)-min(data))*(maxExtrusionWidth-minExtrusionWidth)+minExtrusionWidth
    display(scaledData[:10])
    NUM_POINTS = len(scaledData)
    for i, val in enumerate(scaledData):
        step = fc.polar_to_point(centre=fc.Point(x=0, y=0, z=.1),
                                radius=(outerRad-innerRad)*(i/NUM_POINTS)+innerRad, 
                                angle=i*(turns.to(u.rad).magnitude/NUM_POINTS))
        step.color = [1, 0, 0]
        steps.append(fc.ExtrusionGeometry(width=val, height=.1))
        steps.append(step)
    return steps


In [13]:

stepsL = getSteps(  [d[0] for d in data],
                    turns,
                    OUTER_RADIUS,
                    INNER_RADIUS)

stepsR = getSteps(  [d[1] for d in data],
                    turns,
                    OUTER_RADIUS-MIN_EXTRUSION_WIDTH,
                    INNER_RADIUS-MIN_EXTRUSION_WIDTH)


array([0.93685706, 0.93685704, 0.9368571 , 0.93685703, 0.93685703,
       0.93685708, 0.9368571 , 0.93685718, 0.93685712, 0.93685708])

array([0.99939139, 0.9993914 , 0.99939143, 0.99939144, 0.99939139,
       0.99939143, 0.99939139, 0.99939129, 0.99939142, 0.99939148])

In [14]:
steps = stepsL + stepsR[::-1]

In [15]:
steps[0].width
steps[0].height

0.1

In [16]:
fc.transform(steps[:100],
            'plot',
            fc.PlotControls(color_type='manual',
                            neat_for_publishing=True,
                            zoom=0.7,
                            initialization_data={
                                                 'extrusion_width': steps[0].width, 
                                                 'extrusion_height': steps[0].height}
                            ,tube_type="cylinders"
                                                 )
)

    - use fc.transform(..., controls=fc.PlotControls(style='tube') to disable this message or style='line' for a simpler line preview



In [17]:
gcode = fc.transform(steps, 'gcode')
open('record.gcode', 'w').write(gcode)

   - use fc.transform(..., controls=fc.GcodeControls(printer_name='generic') to disable this message or set it to a real printer name

fc.transform guidance tips are being written to screen if any potential issues are found - hide tips with fc.transform(..., show_tips=False)
tip: set initial `extrusion_width` and `extrusion_height` in the initialization_data to ensure the correct amount of material is extruded:
   - `fc.transform(..., controls=fc.GcodeControls(initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))`



342620